### <ins> Predicciones: Modelo &delta; = 1 </ins>

Vamos a usar el modelo asociado a $\delta$ = 1 para predecir todos los tableros, independientemente del número de pasos que haya entre medias. Para ello, vamos a utilizar este modelo tantas veces como pasos haya entre el tablero inicial y final.

No es un modelo nuevo como tal, pues tan solo estoy usando el modelo que mejor resultados me está dando repetidamente para predecir tableros. Sin embargo, como es otro approach, pues voy a considerarlo aparte.

**1. Importar librerías, definimos las funciones de as métricas y modelos:**

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import tensorflow as tf
from keras.models import load_model
from keras import backend as K


# 1. Define la semilla
SEED = 42  # Según ChatGPT es la mejor
 
# 2. Python built-in random
random.seed(SEED)

# 3. NumPy
np.random.seed(SEED)

# 4. TensorFlow
tf.random.set_seed(SEED)

# (Opcional) Para TensorFlow más determinismo en operaciones GPU:
os.environ['TF_DETERMINISTIC_OPS'] = '1'


def recall_m(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    tp = K.sum(y_true * y_pred_pos)
    possible_positives = K.sum(y_true)
    return tp / (possible_positives + K.epsilon())

def precision_m(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    tp = K.sum(y_true * y_pred_pos)
    predicted_positives = K.sum(y_pred_pos)
    return tp / (predicted_positives + K.epsilon())

def f1_score_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

def specificity_m(y_true, y_pred):
    y_pred_neg = 1 - K.round(K.clip(y_pred, 0, 1))
    y_true_neg = 1 - y_true
    tn = K.sum(y_true_neg * y_pred_neg)
    possible_negatives = K.sum(y_true_neg)
    return tn / (possible_negatives + K.epsilon())

def hamming_loss_m(y_true, y_pred):
    # Fracción de etiquetas incorrectas
    mismatches = K.not_equal(K.round(K.clip(y_pred, 0, 1)), y_true)
    return K.mean(K.cast(mismatches, K.floatx()))


In [2]:
path = 'modelo_delta1'
modelo1 = load_model(
        path, 
        compile=False, 
        custom_objects={
            'accuracy': tf.keras.metrics.BinaryAccuracy(name='accuracy'),
            'recall_m': recall_m,
            'precision_m': precision_m,
            'f1_score_m': f1_score_m,
            'specificity_m': specificity_m,
            'hamming_loss_m': hamming_loss_m})

# Buscamos los paths a todos los excels relevantes:
path = "../Datos"
datos = []
for dirnames,_,filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.xlsx'):
            datos.append(os.path.join(dirnames,filename))

# Cargamos los datos del test:

test = pd.read_excel((datos[0]), sheet_name = 'test', header = 0)
stops = [f'stop.{i}' for i in range(1,401)]

print(datos)

['../Datos\\test.xlsx', '../Datos\\train.xlsx']


#### 2. **Hacemos las predicciones:**

In [ ]:
archivo_excel = "predicciones.xlsx"
with pd.ExcelWriter(archivo_excel, engine='openpyxl') as writer:
    for delta in range(1, 6):
        print(f'δ = {delta}')

        # Filtro los datos y los preparo
        delta_i = test[test['delta'] == delta]
        finales_i = delta_i[stops].values.reshape(-1, 20, 20, 1, order='F')

        # Aplico las predicciones en cadena
        for _ in range(delta):
            finales_i = modelo1.predict(finales_i)

        # Reduzco el array final a filas (1 fila = 1 imagen aplanada de 20x20)
        predicciones = finales_i.reshape(-1, 400).round().astype(np.uint8)

        # Guardo directamente en Excel
        pd.DataFrame(predicciones).to_excel(writer, sheet_name=f"delta_{delta}", index=False)
        print('------------------------------------------------------------------')

δ = 1
312/312 [==============================] - 27s 67ms/step
------------------------------------------------------------------
δ = 2
315/315 [==============================] - 21s 68ms/step
------------------------------------------------------------------
δ = 3
313/313 [==============================] - 21s 67ms/step
------------------------------------------------------------------
δ = 4
307/307 [==============================] - 21s 68ms/step
------------------------------------------------------------------
δ = 5
318/318 [==============================] - 23s 72ms/step
------------------------------------------------------------------


: 